# Week 3 Discussion

## Infographic

* [Fourier Transforms of Images](https://plus.maths.org/content/fourier-transforms-images)

## Links

* [We Recommend a Singular Value Decomposition][svd] -- if you want to understand the math behind the SVD
* [Interactive Guide to the Fourier Transform][fourier] -- if you want to understand the math behind the FFT

[svd]: http://www.ams.org/samplings/feature-column/fcarc-svd
[fourier]: https://betterexplained.com/articles/an-interactive-guide-to-the-fourier-transform/

If you're having trouble getting PIL to work, there's another way you can load an image:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

dog = img.imread("dog.png")
plt.imshow(dog)
plt.show()

In [ ]:
# Automatically display matplotlib plots, so that we don't have to write `plt.show()`.
# Normally this should be in a cell at the top of the notebook.
%matplotlib inline

Matplotlib is a package for plotting, which we'll use throughout the quarter. Note that if you use matplotlib to load the image, you don't need the `as_image()` function from Assignment 2.

In [ ]:
dog.dtype

Depending on which package you use to load an image, pixels may be encoded as integers or floating point (decimal) numbers. The scikit-image package has [some documentation](http://scikit-image.org/docs/dev/user_guide/data_types.html) about what these numbers typically mean.

## Debugging

When you get an error message or an incorrect result:

1. If there's an error message, what does the error message mean?
2. Where (what line) did the error come from? You may have to work backward.
3. Use `print()` or the interactive debugger to inspect variables.

In [ ]:
def add3(x):
    return x + 3

add3("hi")

The error is "must be str, not int" and points to line 2 of the `add3()` function.

We can trace `x` in line 2 back to the parameter `x`. So maybe something is wrong with our call `add3("hi")`.

We can check by adding a print statement:

In [ ]:
def add3(x):
    print(x)
    return x + 3

add3("hi")

We could also use the debugger to check:

In [ ]:
# Load the debugger module. This comes with Jupyter.
import IPython.core.debugger as debug

In [ ]:
def add3(x):
    debug.Tracer()() # note the doubled ()
    return x + 3

add3("hi")

If you're using the terminal, you can instead use:

```python
import ipdb

# To pause the interpreter.
ipdb.set_trace()
```

## Problem Solving: An Example

Break problems into small steps.

If you're trying to figure out how something works, test on small "toy" examples and draw pictures.

In [ ]:
import numpy as np
import skimage as ski
import skimage.color

# Switch from (red, green, blue) to (hue, saturation, value).
# Hue is the color (from red to violet).
# Saturation is how colorful (from colorless to colorful).
# Value is how bright (from black to bright color).
dog_hsv = ski.color.rgb2hsv(dog)

Let's try to find all the green pixels and make them red.

How can we do this? Start by taking a small piece of the dog image, say the lower left corner.

How can we figure out what "green" looks like in HSV?

So we get all pixels with hue "nearby" the grass mean.

How can we define "nearby"?